#### Imports

In [16]:
import torch
import torchvision.transforms as transforms
from torch import nn
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from typing import Dict

import classiq
from classiq import Model, QReg, RX, RY, RZ, synthesize
from classiq.builtin_functions import HardwareEfficientAnsatz
classiq.authenticate()

/home/devilkillerag/document/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/classiq/_internals/authentication/token_manager.py:82: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [5]:
_NUM_QUBITS = 4
_CONNECTIVITY_MAP = "circular"

#### Classical Layer for Image Commpression:
The input MNIST images are all 28 × 28. This Classical Layer will firstly center-crop them to 24 × 24 and
then down-sample them to 4 × 4 for MNIST.

In [18]:
class ClassicalCompressionLayer(nn.Module):
    def __init__(self):
        super(ClassicalCompressionLayer, self).__init__()
        self.center_crop = transforms.CenterCrop((24, 24))
        self.down_sample = transforms.Resize((4, 4))
        self.flatten = nn.Flatten()
        
    def forward(self, x):
        x = self.center_crop(x)
        x = self.down_sample(x)
        x = self.flatten(x)
        return x

#### Quantum Layer for Encoding 
The output of Classical Compression Layer is encoded by this quantum layer into a quantum circuit. We use Angle encoding to encode 4 pixels per qubit using RX, RY, RZ, and RX gate on each qubit.

In [19]:
class QuantumEncodingLayer(Model):
    def __init__(self):
        super().__init__()
        
    def encode_pixels(self, pixel_values: torch.Tensor) -> Dict[str, QReg]:
        # Split pixel values into groups of 4
        pixel_groups = pixel_values.split(4)
        
        # Initialize dictionary to store qubit outputs
        qubit_outputs = {}
        
        # Encode each group of 4 pixels into angles for RX, RY, RZ, RX gates
        for i, pixel_group in enumerate(pixel_groups):
            rx_angle = pixel_group[0] * (2 * torch.pi / 255)
            ry_angle = pixel_group[1] * (2 * torch.pi / 255)
            rz_angle = pixel_group[2] * (2 * torch.pi / 255)
            rx2_angle = pixel_group[3] * (2 * torch.pi / 255)
            
            # Apply gates to corresponding qubit
            qubit_outputs[f"qubit_{i}"] = RX(rx_angle) & RY(ry_angle) & RZ(rz_angle) & RX(rx2_angle)
            
        return qubit_outputs

#### Quantum Layer for Entanglement 

In [20]:
class QuantumEntanglementLayer(Model):
    def __init__(self):
        super().__init__()

    def add_entanglement_layer(self) -> Dict[str, QReg]:
        hwea_params = HardwareEfficientAnsatz(
            num_qubits=_NUM_QUBITS,
            connectivity_map=_CONNECTIVITY_MAP,
            one_qubit_gates=[],
            two_qubit_gates=["rzz, rxx, rzx, cz"],
        )
        return self.HardwareEfficientAnsatz(hwea_params)

#### Hybrid Quantum Neural Network

In [21]:
class HybridQuantumNeuralNetwork(Model):
    def __init__(self):
        super().__init__()

        # Instantiate Layer
        self.classical_compression_layer = ClassicalCompressionLayer()
        self.encoding_layer = QuantumEncodingLayer()
        self.entanglement_layer = QuantumEntanglementLayer()

        # Import Data
        
    
        # Add Encoding Layer
        encoding_out = self.encoding_layer.encode_pixels()

        # Add Entanglement Layer
        entanglement_out = self.entanglement_layer.add_rzz_layer()

        # Add layers to the model
        self.add(encoding_out, entanglement_out)

    def forward(self, x):
        # Classical Compression
        compressed_data = self.classical_compression_layer(x)

        # Quantum Encoding
        encoding_result = self.encoding_layer(compressed_data)

        # Quantum Entanglement
        entanglement_result = self.entanglement_layer()

        # Concatenate quantum and classical outputs
        output = self.concatenate(
            [encoding_result, entanglement_result, compressed_data]
        )

        return output

In [1]:
# hybrid_model = HybridQuantumNeuralNetwork()
# quantum_program = synthesize(hybrid_model.get_model())

##### Above implementation was wrong: Re-trying: Take 3

In [ ]:
import torch
import classiq

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from typing import Dict
from classiq import Model, synthesize, QReg, QFunc
from classiq.builtin_functions import HardwareEfficientAnsatz
from classiq.applications.qnn import QLayer
from classiq.execution import execute_qnn
from classiq.synthesis import SerializedQuantumProgram

from classiq.applications.qnn.types import (
    MultipleArguments,
    SavedResult,
    ResultsCollection,
)

# classiq.authenticate()

In [ ]:
# constants
_NUM_QUBITS =  4
_REPS = 1
_FULLY_CONNECTED_MESH = [[0, 1], [1, 2], [2, 3], [3, 0]]
_LEARNING_RATE = 1.0

In [ ]:
def add_entanglement(md: Model, prefix: str, in_wire=None) -> Dict[str, QReg]:
    if in_wire is not None:
        kwargs = { "in_wires": { "IN": in_wire["OUT"] } }
    else: 
        kwargs = {}
    
    hwea_params = HardwareEfficientAnsatz(
        num_qubits=_NUM_QUBITS,
        connectivity_map=_FULLY_CONNECTED_MESH,
        reps=_REPS,
        one_qubit_gates=[],
        two_qubit_gates=["rzz", "rxx", "rzx"],
        parameter_prefix=prefix,
    )
    
    return md.HardwareEfficientAnsatz(hwea_params, **kwargs)

In [ ]:
model = Model()
out1 = add_entanglement(model, "input_")
out2 = add_entanglement(model, "weight_", out1)

quantum_program = synthesize(model.get_model())

In [ ]:
def execute(quantum_program: SerializedQuantumProgram, arguments: MultipleArguments) -> ResultsCollection:
    return execute_qnn(quantum_program, arguments)

In [ ]:
# TODO: MODIFY THIS

# Post-process the result, returning a dict:
# Note: this function assumes that we only care about
#   differentiating a single state (|0>)
#   from all the rest of the states.
#   In case of a different differentiation, this function should change.
def post_process(result: SavedResult) -> torch.Tensor:
    """
    Take in a `SavedResult` with `ExecutionDetails` value type, and return the
    probability of measuring |0> which equals the amount of `|0>` measurements
    divided by the total amount of measurements.
    """
    counts: dict = result.value.counts
    # The probability of measuring |0>
    p_zero: float = counts.get("0", 0.0) / sum(counts.values())
    return torch.tensor(p_zero)